# Vertikale Datenbank für Events - Invertierter Index

Im folgenden wird gezeigt, wie aus einer einfachen Sequenz eine Vertikale Datenbank erstellt wird.

In [1]:
using LogClustering: Index

In [16]:
#           A B C E D G E A B C F E E A B D
sequence = [1,2,3,5,4,7,5,1,2,3,6,5,5,1,2,4]
#                             1 1 1 1 1 1 1
#           1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6

vertical = Index.invert(sequence)

Dict{Int64,Array{Int64,1}} with 7 entries:
  7 => [6]
  4 => [5, 16]
  2 => [2, 9, 15]
  3 => [3, 10]
  5 => [4, 7, 12, 13]
  6 => [11]
  1 => [1, 8, 14]

In einer vertikalen Datenbank werden jedem Event, dessen Auftritte Zugeordnet.

# Vertikale Datenbank für Episoden

Analog zu einem inverttierten Index für Events kann dieser genutzt werden, um die Auftritte von Episoden vorzuhalten.

In [3]:
events = keys(vertical)
episodes = map(event -> [event], events)
occurrences = map(event -> vertical[event], events)

7-element Array{Array{Int64,1},1}:
 [6]           
 [5, 16]       
 [2, 9, 15]    
 [3, 10]       
 [4, 7, 12, 13]
 [11]          
 [1, 8, 14]    

In [4]:
oSet = Dict(map(eo -> eo[1] => map(o -> o:o, eo[2]), zip(episodes, occurrences)))

Dict{Array{Int64,1},Array{UnitRange{Int64},1}} with 7 entries:
  [6] => UnitRange{Int64}[11:11]
  [2] => UnitRange{Int64}[2:2, 9:9, 15:15]
  [3] => UnitRange{Int64}[3:3, 10:10]
  [5] => UnitRange{Int64}[4:4, 7:7, 12:12, 13:13]
  [4] => UnitRange{Int64}[5:5, 16:16]
  [1] => UnitRange{Int64}[1:1, 8:8, 14:14]
  [7] => UnitRange{Int64}[6:6]

Eine Vertikale Datenbank für Episoden der Länge eins, enthät nicht mehr Information, als die Vertikale Datenbank für Events, allerdings können damit eben auch längere Episoden dargestellt werden, wie:

In [5]:
# [1,2,3,5,4,7,5,1,2,3,6,5,5,1,2,4]
#                    1 1 1 1 1 1 1
#  1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6

In [8]:
oSet[[1,2,5]] = [1:4, 1:7, 1:12, 1:13, 8:12, 8:13]
oSet

Dict{Array{Int64,1},Array{UnitRange{Int64},1}} with 8 entries:
  [6]       => UnitRange{Int64}[11:11]
  [2]       => UnitRange{Int64}[2:2, 9:9, 15:15]
  [3]       => UnitRange{Int64}[3:3, 10:10]
  [5]       => UnitRange{Int64}[4:4, 7:7, 12:12, 13:13]
  [4]       => UnitRange{Int64}[5:5, 16:16]
  [1]       => UnitRange{Int64}[1:1, 8:8, 14:14]
  [1, 2, 5] => UnitRange{Int64}[1:4, 1:7, 1:12, 1:13, 8:12, 8:13]
  [7]       => UnitRange{Int64}[6:6]

Diese Darstellung ist zwar eindeutig, allerdings hat sie zwei Nachteile:

1. Zum einen überschneiden sich die im `oSet` enthaltenen Episoden.

2. Zum anderen ist nicht klar, an welcher Position, die Events zwischen der Startposition `T_s` und Endposition `T_e` eines Intervalls auftreten.

Es ist daher wünschenswert lediglich die minimalen Auftritte einer Episode zu bestimmen, um keine Überschneidenungen, außer an der Startposition oder den Endpositionen zu haben.